## 訓練預測模型
在這個筆記本中，我們將訓練一個深度學習模型，該模型基於歷史價格數據，學習下一個交易日是否會觸及目標價格或止損價格，以進行多頭/空頭交易。

模型：
* 模型名稱：model_long_short_predict
* 多層感知器（MLP）（前向神經網絡）
* 3層：輸入、隱藏、輸出
* 二元分類
* 輸入：收盤價，SMA（2到16），ROC（2到16）
* 輸出：在接下來的五天內，多頭或空頭交易是否觸及了2%的利潤目標，而未觸及1.5%的止損點？

In [ ]:
%run ./init_model.py 'model_long_short_predict'

## 步驟一： 從 S3 獲取 Day26 建立的資料

In [ ]:
# get S3 bucket
s3bucket=!(aws s3 ls | grep algotrading- | awk  '{print $3}')
s3bucket=s3bucket[0]
s3bucket

In [ ]:
import sys
!{sys.executable} -m pip install PyAthena

In [ ]:
import os
import sagemaker as sage
from sagemaker import get_execution_role
import datetime
from sagemaker.tensorflow import TensorFlow
import json

role = get_execution_role()
sess = sage.Session()
region = sess.boto_session.region_name

In [ ]:
import pandas as pd
from pyathena import connect
conn = connect(s3_staging_dir='s3://'+s3bucket+'/results/',
               region_name=region)

df = pd.read_sql("SELECT * FROM algo_data.hist_data_daily;", conn)
df.set_index(pd.DatetimeIndex(df['dt']),inplace=True)
del df['dt']
df.head()

In [ ]:
df.to_csv('local/'+model_name+'/input/data/training/data_orig.csv')
print("count=%s" % len(df))
df.head()

In [ ]:
%matplotlib notebook
df["close"].plot()

## 步驟二：本機端（Notebook Instance環境）資料前置處理

### 撰寫資料前置處理程式碼
這段程式碼主要是用於進行金融市場數據的預處理，以生成用於機器學習或其他分析模型的輸入數據。下面是程式碼的主要步驟和功能：

匯入必要的Python庫：
numpy（作為 np 別名）：用於數字運算。
pandas（作為 pd 別名）：用於數據處理和分析。
talib（作為 ta 別名）：用於技術分析指標的計算。
math：用於一些數學運算。

使用Pandas的read_csv函數讀取CSV文件 data_orig_file，並將其存儲在DataFrame d 中。同時，該函數也將日期時間列 dt 設置為索引列。接著，打印了 d 的前幾行數據以檢查。
設置了一些變數，包括 repeatCount、repeatStep、lookBack、forwardWindow、profitTarget 和 stopTarget 等，這些變數將在後續的計算中使用。

創建了列名為 hData 的列表，其中包含了即將創建的DataFrame的列名。這些列包括日期時間 dt、收盤價 close、一系列移動平均值（sma）和一系列變化率（roc），以及 long 和 short 兩個列。
創建了一個空的 tData 列表，用於存儲處理後的數據。
開始對數據進行迴圈處理，其中包括以下步驟：

提取日期時間和收盤價。
計算移動平均值（SMA）和變化率（ROC）。
計算特定條件下的 long 和 short 信號，並將它們添加到 inputRec 中。
將處理好的 inputRec 添加到 tData 列表中。
輸出 lCount 和 sCount，這些變數分別表示 long 和 short 信號的計數。

創建一個新的DataFrame df1，並將處理後的數據存儲在其中。同時，將日期時間列 dt 設置為索引列，並刪除了原本的 dt 列。
最後，將處理好的數據存儲為一個新的CSV文件 data_file，同時打印 df1 的前幾行數據和數據的總數。這個新的CSV文件包含了經過處理的金融市場數據，可以用於後續的分析或機器學習模型的訓練。

In [ ]:
%mkdir model

In [ ]:
%%writefile model/{model_name}_prep.py
#!/usr/bin/env python

import numpy as np
import pandas as pd
import talib as ta
from talib.abstract import *
import math

prefix = '/opt/ml/'
input_path = prefix + 'input/data/training'

data_orig_file = input_path+'/data_orig.csv'
data_file = input_path+'/data.csv'

d = pd.read_csv(data_orig_file,infer_datetime_format=True, parse_dates=['dt'], index_col=['dt'])
print(d.head())

repeatCount=15
repeatStep=1
lookBack=repeatCount*repeatStep
forwardWindow=5

profitTarget=2.0/100.0
stopTarget=1.5/100.0

iCount=lookBack

# header
hData=["dt"]
hData.append("close")
for a in range(0,repeatCount):
    hData.append("sma"+str((a+2)*repeatStep))
for a in range(0,repeatCount):
    hData.append("roc"+str((a+2)*repeatStep))
hData.append("long")
hData.append("short")

# data
tData=[]

inputs = {
    'close': np.array(d["close"])
}
sma=[]
for a in range(0,repeatCount):
    sma.append(SMA(inputs,timeperiod=(a+1)*repeatStep+1))
roc=[]
for a in range(0,repeatCount):
    roc.append(ROC(inputs,timeperiod=(a+1)*repeatStep+1))

closeList=d["close"]
dLen=len(d)
n=0
lCount=0
sCount=0
nCount=0
n=0
for idx,row in d.iterrows():
    if n<dLen-forwardWindow-1:
        dt1=idx
        cl=row["close"]
        inputRec=[]
        inputRec.append(idx)

        inputRec0=[]

        #close
        inputRec0.append(cl)

        #sma
        for a in range(0,repeatCount):
            if math.isnan(sma[a][n]):
                inputRec0.append(cl)
            else:
                inputRec0.append(sma[a][n])

        m1=min(inputRec0)
        m2=max(inputRec0)
        for a in inputRec0:
            if m2-m1==0:
                inputRec.append(0)
            else:
                inputRec.append((a-m1)/(m2-m1))

        #roc
        for a in range(0,repeatCount):
            if math.isnan(roc[a][n]):
                inputRec.append(0)
            else:
                inputRec.append(roc[a][n])

        rClose=closeList[n+1:min(dLen-1,n+1+forwardWindow)].values.tolist()
        low=min(rClose)
        high=max(rClose)
        
        #long
        long=0
        if high>=cl+cl*profitTarget and low>=cl-cl*stopTarget:
            long=1
            lCount=lCount+1
        inputRec.append(long)
 
        #short
        short=0
        if low<=cl-cl*profitTarget and high<=cl+cl*stopTarget:
            short=1
            sCount=sCount+1
        inputRec.append(short)

        tData.append(inputRec)
        n=n+1
          
print("lCount=%s,sCount=%s" % (lCount,sCount))
df1=pd.DataFrame(tData,columns=hData)
df1.set_index(pd.DatetimeIndex(df1['dt']), inplace=True)
del df1['dt']
 
df1.to_csv(data_file)
print(df1.head())
print("count=%s" % (len(df1)))

### 本機端執行上面寫的程式碼 in a Docker Container

In [ ]:
!cp model/{model_name}_prep.py model/train
!chmod 777 model/train
!docker build -t {model_name}_prep .
!docker run -v $(pwd)/local/$model_name:/opt/ml --rm {model_name}_prep train

### 產生訓練 & 測試資料

In [ ]:
df = pd.read_csv("local/"+model_name+"/input/data/training/data.csv", infer_datetime_format=True, parse_dates=['dt'], index_col=['dt'])
print("totalCount=%s" % len(df))

trainCount=int(len(df)*0.4)
dfTrain = df.iloc[:trainCount]
dfTrain.to_csv("local/"+model_name+"/input/data/training/data_train.csv")
print("trainCount=%s" % len(dfTrain))

dfTest = df.iloc[trainCount:]
dfTest.to_csv("local/"+model_name+"/input/data/training/data_test.csv")
print("testCount=%s" % len(dfTest))
dfTest.head()

## 步驟3: 訓練你的模型
這邊我們保留原 blog 的程式碼來訓練模型

In [ ]:
%%writefile model/{model_name}.py
#!/usr/bin/env python
from __future__ import print_function

import os
import sys
import traceback
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

yLen=2
b=0

# Optional
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# These are the paths to where SageMaker mounts interesting things in your
# container.
prefix = '/opt/ml/'

input_path = prefix + 'input/data/training/data_train.csv'
test_path = prefix + 'input/data/training/data_test.csv'

output_path = os.path.join(prefix, 'output')
model_path = os.path.join(prefix, 'model')

# Process and prepare the data
def data_process(df):
    global yLen
    global b
    dataX=[]
    dataY=[]
    for idx,row in df.iterrows():
        row1=[]
        r=row[1:len(row)-yLen]
        for a in r:
            row1.append(a)
        x=np.array(row1)
        y=np.array(row[len(row)-yLen:])
        b=len(x)
        dataX.append(x)
        dataY.append(y)
    dataX=np.array(dataX).astype(np.float32)
    dataY=np.array(dataY).astype(np.float32)
    return dataX,dataY,b

def build_classifier():
    global b
    global yLen
    print("build_classifier:b=%s,yLen=%s" % (b,yLen))
    model = Sequential()
    model.add(Dense(b, input_dim=b, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(int(b/2), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(yLen,kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def generate_model(dataX, dataY, b):
    model=build_classifier()
    model.fit(dataX, dataY, epochs=100, batch_size=1)
    scores = model.evaluate(dataX, dataY, verbose=0)
    print("Training Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return model
        
def train():
    print('Starting the training.')
    try:
        raw_data = pd.read_csv(input_path)
        #print(raw_data)
        X, y, b = data_process(raw_data)
        model = generate_model(X, y, b)
        model.save(os.path.join(model_path, 'model.h5'))
        
        print('Training is complete. Model saved.')
        
        raw_data = pd.read_csv(test_path)
        testX, testY, b = data_process(raw_data)
        scores = model.evaluate(testX, testY, verbose=0)
        print("Test Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
    except Exception as e:
        # Write out an error file. This will be returned as the failure
        # Reason in the DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(output_path, 'failure'), 'w') as s:
            s.write('Exception during training: ' + str(e) + '\n' + trc)
        # Printing this causes the exception to be in the training job logs
        print(
            'Exception during training: ' + str(e) + '\n' + trc,
            file=sys.stderr)
        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)

if __name__ == '__main__':
    train()

    # A zero exit code causes the job to be marked a Succeeded.
    sys.exit(0)

### 本地端訓練（用 SageMaker 啟動的 Notebook Instance 上面的 docker 進行訓練工作）
最後會輸出 .h5 檔就完成我們可以用來預測的 model file 了

In [ ]:
# Build Local ML Image
!cp model/{model_name}.py model/train
!chmod 777 model/train
!docker build -t {model_name} .
!docker run -v $(pwd)/local/$model_name:/opt/ml --rm {model_name} train

In [ ]:
!ls -la local/{model_name}/model/model.h5
!cp local/{model_name}/model/model.h5 ./model/{model_name}.h5
!ls -la ./model/model_*.h5